In [1]:
import torch
import numpy as np
import pgnn_control as pgnnc
from robot_env import robot_env
from utils import to_tensors, combine_state, wrap_to_pi, rotate
import os
# cwd = os.path.dirname(os.path.realpath(__file__))
from print_xacros import get_names, compile_to_urdf
from apply_policy3 import apply_policy, make_goal_memory
from planning_utils import compare_velocities

all_name_list = get_names()
print(len(all_name_list), ' designs')


# divide them up into seen and unseen
unseen_inds = []
seen_inds = []
for i_urdf in range(len(all_name_list)):
    urdf = all_name_list[i_urdf]
    if (urdf[0:3]==urdf[3:6][::-1]):
        seen_inds.append(i_urdf)
    else:
        unseen_inds.append(i_urdf)
print(len(seen_inds), ' seen designs')
print(len(unseen_inds), ' unseen designs')




144  designs
12  seen designs
132  unseen designs


In [2]:
device = torch.device('cpu')


# make some direction goals
goal_memory = make_goal_memory(41) # 10*4 + 1

# load up a learned policy to test
# # load previous weights if desired
folder = 'mbrl_v8_test9' # BEST
folder = 'mbrl_v8_test9NT2' 
control_fname = os.path.join(folder, 'multidesign_control_iter3.pt')
if os.path.exists(control_fname):
    print('Loading weights from ' + control_fname)
    save_dict = torch.load( control_fname, map_location=lambda storage, loc: storage)

    gnn_nodes_control = pgnnc.create_GNN_nodes(save_dict['internal_state_len'], 
                save_dict['message_len'], save_dict['hidden_layer_size'], 
                device, goal_len=3, body_input= True) 

    pgnnc.load_state_dicts(gnn_nodes_control, save_dict['gnn_state_dict'])



Loading weights from mbrl_v8_test9NT2/multidesign_control_iter3.pt


In [8]:
urdf_names = all_name_list
# random_choice = np.random.choice(len(unseen_inds), 8*6, replace=False)
# random_choice = np.random.choice(len(unseen_inds), 4*3, replace=False)
# urdf_names = [all_name_list[unseen_inds[ind]] for ind in random_choice]

# selected for best, middle, worst
# urdf_names =['wnwwlw', 'wwwwnw', 'lnwwll', 'wwwwll', 
#              'llwlwl', 'wwllnw', 'lnwlwl', 'llllww', 
#              'lnlwwl', 'wnwlnl', 'lnllwl', 'lwllnl']
# urdf_names = [all_name_list[ind] for ind in unseen_inds]

print(urdf_names)

# urdf_names = ['llllll', 'wnwwnw', 'llwwll', 'lnwwnl']
# urdf_names += [ 'lnllnl', 'lwllwl', 'lwwwwl', 'wlwwlw', 
#                'wwllww', 'wwwwww', 'wnllnw', 'wllllw']

# if 'llllll' in urdf_names:
#     data_subfolder = os.path.join(folder,'data_seen')
# else:
data_subfolder = os.path.join(folder,'zero_shot_data')

if not(os.path.exists(data_subfolder)):
    os.mkdir(data_subfolder)
    print('Created folder ' + data_subfolder)
else:
    print('Using folder ' + data_subfolder)
    


CREATE_VIDEOS= False
# CREATE_VIDEOS= True
for i_urdf in range(len(urdf_names)):
    urdf_name = urdf_names[i_urdf]
    urdf_file =   'urdf/' + urdf_name  + '.urdf'
    if not(os.path.exists(urdf_file)):
        compile_to_urdf(urdf_name)

['llllll', 'lllllw', 'llllwl', 'llllww', 'llllnl', 'llllnw', 'lllwll', 'lllwlw', 'lllwwl', 'lllwww', 'lllwnl', 'lllwnw', 'llwlll', 'llwllw', 'llwlwl', 'llwlww', 'llwlnl', 'llwlnw', 'llwwll', 'llwwlw', 'llwwwl', 'llwwww', 'llwwnl', 'llwwnw', 'lwllll', 'lwlllw', 'lwllwl', 'lwllww', 'lwllnl', 'lwllnw', 'lwlwll', 'lwlwlw', 'lwlwwl', 'lwlwww', 'lwlwnl', 'lwlwnw', 'lwwlll', 'lwwllw', 'lwwlwl', 'lwwlww', 'lwwlnl', 'lwwlnw', 'lwwwll', 'lwwwlw', 'lwwwwl', 'lwwwww', 'lwwwnl', 'lwwwnw', 'lnllll', 'lnlllw', 'lnllwl', 'lnllww', 'lnllnl', 'lnllnw', 'lnlwll', 'lnlwlw', 'lnlwwl', 'lnlwww', 'lnlwnl', 'lnlwnw', 'lnwlll', 'lnwllw', 'lnwlwl', 'lnwlww', 'lnwlnl', 'lnwlnw', 'lnwwll', 'lnwwlw', 'lnwwwl', 'lnwwww', 'lnwwnl', 'lnwwnw', 'wlllll', 'wllllw', 'wlllwl', 'wlllww', 'wlllnl', 'wlllnw', 'wllwll', 'wllwlw', 'wllwwl', 'wllwww', 'wllwnl', 'wllwnw', 'wlwlll', 'wlwllw', 'wlwlwl', 'wlwlww', 'wlwlnl', 'wlwlnw', 'wlwwll', 'wlwwlw', 'wlwwwl', 'wlwwww', 'wlwwnl', 'wlwwnw', 'wwllll', 'wwlllw', 'wwllwl', 'wwllww',

In [9]:
T = 20
vel_metric_list = []
vel_baseline_list = []
video_names = []
for i_urdf in range(len(urdf_names)):
    urdf = urdf_names[i_urdf]

    save_path = os.path.join(data_subfolder, urdf + '_apply_policy.ptx')
    video_name = os.path.join(data_subfolder, urdf + '_goal')
    video_names.append(video_name)
    if CREATE_VIDEOS:
        apply_policy(urdf, goal_memory,
          gnn_nodes_control, device, save_path, show_GUI=CREATE_VIDEOS, 
                 video_name = video_name, sim_speed_factor=10, overhead=False)
    else:
        apply_policy(urdf, goal_memory, 
                     gnn_nodes_control, device, save_path, show_GUI=False)

    save_dict = torch.load(save_path, map_location=lambda storage, loc: storage)
    vel_metric, vel_metric_baseline = compare_velocities(
            save_dict['states_memory'],
            save_dict['goal_memory'], 
            save_dict['run_lens'],
            10, T )
    vel_metric_list.append(vel_metric)
    vel_baseline_list.append(vel_metric_baseline)
    print(urdf + ': ' + str(np.round(vel_metric,2))
          + ' baseline ' + str(np.round(vel_metric_baseline,2))
          + ' ' + str(i_urdf) + '/' + str(len(urdf_names)))

llllll: 0.04 baseline 0.12 0/144
lllllw: 0.04 baseline 0.12 1/144
llllwl: 0.04 baseline 0.12 2/144
llllww: 0.05 baseline 0.12 3/144
llllnl: 0.05 baseline 0.12 4/144
llllnw: 0.04 baseline 0.12 5/144
lllwll: 0.04 baseline 0.12 6/144
lllwlw: 0.05 baseline 0.12 7/144
lllwwl: 0.04 baseline 0.12 8/144
lllwww: 0.06 baseline 0.12 9/144
lllwnl: 0.05 baseline 0.12 10/144
lllwnw: 0.05 baseline 0.12 11/144
llwlll: 0.04 baseline 0.12 12/144
llwllw: 0.04 baseline 0.12 13/144
llwlwl: 0.05 baseline 0.12 14/144
llwlww: 0.05 baseline 0.12 15/144
llwlnl: 0.05 baseline 0.12 16/144
llwlnw: 0.04 baseline 0.12 17/144
llwwll: 0.03 baseline 0.12 18/144
llwwlw: 0.03 baseline 0.12 19/144
llwwwl: 0.04 baseline 0.12 20/144
llwwww: 0.04 baseline 0.12 21/144
llwwnl: 0.03 baseline 0.12 22/144
llwwnw: 0.04 baseline 0.12 23/144
lwllll: 0.05 baseline 0.12 24/144
lwlllw: 0.04 baseline 0.12 25/144
lwllwl: 0.02 baseline 0.12 26/144
lwllww: 0.03 baseline 0.12 27/144
lwllnl: 0.06 baseline 0.12 28/144
lwllnw: 0.05 baseline 0.

In [10]:

if len(urdf_names)>50:
    vel_data_path = os.path.join(data_subfolder, 'transfer_results.ptx')
    vel_dict = dict()
    vel_dict['unseen_inds'] = unseen_inds
    vel_dict['seen_inds'] = seen_inds
    vel_dict['urdf_names'] = urdf_names
    vel_dict['vel_metric_list'] = vel_metric_list
    vel_dict['vel_baseline_list'] = vel_baseline_list
    torch.save(vel_dict, vel_data_path)

    vel_save_path = os.path.join(data_subfolder, 'transfer_results.csv')
    seen_names = [urdf_names[s] for s in seen_inds]
    unseen_names = [urdf_names[s] for s in unseen_inds]

    vel_metric_list = np.array(vel_metric_list)
    vel_baseline_list = np.array(vel_baseline_list)

    with open(vel_save_path, 'w') as fp:
        names_text = ''
        for urdf in unseen_names:
            names_text = names_text + urdf + ',' 

        fp.write('--- Unseen Names: ---\n')
        fp.write(names_text + '\n')
        fp.write('Metric Mean: ' + str(np.mean(vel_metric_list[unseen_inds]))+'\n')
        fp.write('Metric Min: ' + str(np.min(vel_metric_list[unseen_inds]))+'\n')
        fp.write('Metric Max: ' + str(np.max(vel_metric_list[unseen_inds]))+'\n')
        fp.write('Metric Rescaled: ' + str(
            np.mean( (vel_baseline_list[unseen_inds] - vel_metric_list[unseen_inds])
                    /vel_baseline_list[unseen_inds])
            )+'\n')

        fp.write('--- Metric: ---\n')
        np.savetxt(fp, vel_metric_list[unseen_inds], delimiter=',')   
        fp.write('--- Baseline: ---\n')
        np.savetxt(fp, vel_baseline_list[unseen_inds], delimiter=',')

        names_text = ''
        for urdf in seen_names:
            names_text = names_text + urdf + ',' 

        fp.write('--- Seen Names: ---\n')
        fp.write(names_text + '\n')
        fp.write('Metric Mean: ' + str(np.mean(vel_metric_list[seen_inds]))+'\n')
        fp.write('Metric Min: ' + str(np.min(vel_metric_list[seen_inds]))+'\n')
        fp.write('Metric Max: ' + str(np.max(vel_metric_list[seen_inds]))+'\n')
        fp.write('Metric Rescaled: ' + str(
            np.mean( (vel_baseline_list[seen_inds] - vel_metric_list[seen_inds])
                    /vel_baseline_list[seen_inds])
            )+'\n')
        fp.write('--- Metric: ---\n')
        np.savetxt(fp, vel_metric_list[seen_inds], delimiter=',')   
        fp.write('--- Baseline: ---\n')
        np.savetxt(fp, vel_baseline_list[seen_inds], delimiter=',')
        print('wrote file')

wrote file


In [8]:
video_names

['mbrl_v8_test9/zero_shot_data/wnwwlw_goal',
 'mbrl_v8_test9/zero_shot_data/wwwwnw_goal',
 'mbrl_v8_test9/zero_shot_data/lnwwll_goal',
 'mbrl_v8_test9/zero_shot_data/wwwwll_goal',
 'mbrl_v8_test9/zero_shot_data/llwlwl_goal',
 'mbrl_v8_test9/zero_shot_data/wwllnw_goal',
 'mbrl_v8_test9/zero_shot_data/lnwlwl_goal',
 'mbrl_v8_test9/zero_shot_data/llllww_goal',
 'mbrl_v8_test9/zero_shot_data/lnlwwl_goal',
 'mbrl_v8_test9/zero_shot_data/wnwlnl_goal',
 'mbrl_v8_test9/zero_shot_data/lnllwl_goal',
 'mbrl_v8_test9/zero_shot_data/lwllnl_goal']

In [9]:
# # # video tile compile. This works ok
import subprocess
tag = 'output'
tag = 'bestMidWorst'
for i in range(6):
    end_str = str(i) + '.mp4' # add the goal number and filetype to the video names
    
    # get the number of frames from the video segments
    print('getting video lengths')
    vid_frames = []
    for vid in video_names:
        batcmd = ('ffprobe -select_streams v -show_streams '
               + vid+end_str)
        result = str(subprocess.check_output(batcmd, shell=True))
        nb_frames_index = result.find('nb_frames=')
        next_backslash = result[nb_frames_index:].find('\\n')
        n_frames = result[nb_frames_index+len('nb_frames='):
                          nb_frames_index+next_backslash]
        vid_frames.append(int(n_frames))

    # calculate retiming so that all segments can be retimed to be the same length
    print('relative retiming: ' + str(np.array(vid_frames)/np.max(vid_frames)))
    retiming = 3*np.max(vid_frames)/np.array(vid_frames) # slow down by 3x (about realtime)
    
    # tile videos side by side (2x2)
#     run_str = ('ffmpeg'+
#                ' -i '+ video_names[0]+end_str+
#                ' -i '+ video_names[1]+end_str+ 
#                ' -i '+ video_names[2]+end_str+
#                ' -i '+ video_names[3]+end_str+
#                ' -filter_complex '+ 
#                '"nullsrc=size=2048x1536 [base]; '+
#                '[0:v] setpts=' + str(retiming[0]) +'*PTS, scale=1024x768 [upperleft]; '+
#                '[1:v] setpts=' + str(retiming[1]) +'*PTS, scale=1024x768 [upperright]; '+
#                '[2:v] setpts=' + str(retiming[2]) +'*PTS, scale=1024x768 [lowerleft]; '+
#                '[3:v] setpts=' + str(retiming[3]) +'*PTS, scale=1024x768 [lowerright]; '+
#                '[base][upperleft] overlay=shortest=1 [tmp1]; '+
#                '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
#                '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
#                '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768" ' +
#                os.path.join(data_subfolder,'output'+end_str)
#               )
    
    # tile videos side by side (4x3)
    run_str = 'ffmpeg'
    nv = len(video_names)
    
    # many videos
    n_rows = 3*2
    n_cols = 4*2
    
    # 12 videos
    n_rows = 3
    n_cols = 4
    
    for ii in range(nv):
        video_name = video_names[ii]
        run_str +=  ( ' -i ' + video_name+end_str)
    run_str += ' -filter_complex " '
    for ii in range(nv):
#         run_str += '[' +str(ii) +':v] setpts=' + str(retiming[ii]) +'*PTS, scale=1024x768 [a'+str(ii)+']; '
        run_str += '[' +str(ii) +':v] setpts=' + str(retiming[ii]) +'*PTS, scale=512x384 [a'+str(ii)+']; '

    for ri in range(n_rows):
        for ci in range(n_cols):
            run_str += '[a' + str(ci+ ri*n_cols) + ']'
        run_str += 'hstack=inputs='+str(n_cols)+'[r' + str(ri) + '];'
    for ri in range(n_rows):
        run_str +='[r' + str(ri) + ']'
    run_str += 'vstack=inputs='+str(n_rows)+'[out]"'
            
    run_str += ' -map "[out]" '
    run_str += os.path.join(data_subfolder,tag+end_str) 
    run_str = run_str + ' -y'
    print('Running tile for goal ' + str(i) )
    status = os.system(run_str)
    print('Status (0 is good):' + str(status))
    
print('done')

getting video lengths
relative retiming: [0.89795918 0.91836735 0.7755102  0.96938776 0.7755102  0.63265306
 0.86734694 0.97959184 0.84693878 0.63265306 1.         0.74489796]
Running tile for goal 0
Status (0 is good):0
getting video lengths
relative retiming: [1.         0.98198198 0.72972973 0.76576577 0.78378378 0.74774775
 0.74774775 0.85585586 0.96396396 0.68468468 0.71171171 0.81981982]
Running tile for goal 1
Status (0 is good):0
getting video lengths
relative retiming: [0.62831858 0.98230088 0.72566372 0.7079646  0.54867257 0.7699115
 0.72566372 0.87610619 0.55752212 0.54867257 1.         0.88495575]
Running tile for goal 2
Status (0 is good):0
getting video lengths
relative retiming: [1.         0.86029412 0.56617647 0.64705882 0.67647059 0.59558824
 0.61029412 0.74264706 0.5        0.58823529 0.65441176 0.5       ]
Running tile for goal 3
Status (0 is good):0
getting video lengths
relative retiming: [0.5826087  1.         0.64347826 0.66956522 0.79130435 0.91304348
 0.8     

In [23]:
# write text overlay on video

direction_str = ['Direction →']
direction_str += ['Direction ↑'] 
direction_str += ['Direction ←']
direction_str += ['Direction ↓']
direction_str += ['Turn in place ↰']
direction_str += ['Turn in place ↲']
# [right up left forward turnleft turnright]


# ffmpeg -i input.mp4 -vf drawtext="fontfile=/path/to/font.ttf: \
# text='Stack Overflow': fontcolor=white: fontsize=24: box=1: boxcolor=black@0.5: \
# boxborderw=5: x=(w-text_w)/2: y=(h-text_h)/2" -codec:a copy output.mp4
# for i in range(6):
for i in [4,5]:
    end_str = str(i) + '.mp4' # add the goal number and filetype to the video names
    
    run_str = 'ffmpeg -y -i '
    run_str += os.path.join(data_subfolder,tag+end_str)
    run_str += ' -vf drawtext="fontsize=50:fontfile=FreeSerif.ttf:'
    run_str += "text=''"+direction_str[i]+"'':"
    # run_str += 'x=(w-text_w)/2:y=(h-text_h)/2"'
    run_str += 'x=(w-text_w)/2:y=5" '
    run_str += os.path.join(data_subfolder, tag+'_txt'+end_str)
    print(run_str)
    status = os.system(run_str)
    print('Status (0 is good):' + str(status))
print('Done')

ffmpeg -y -i mbrl_v8_test9/zero_shot_data/bestMidWorst4.mp4 -vf drawtext="fontsize=50:fontfile=FreeSerif.ttf:text=''Turn in place ↰'':x=(w-text_w)/2:y=5" mbrl_v8_test9/zero_shot_data/bestMidWorst_txt4.mp4
Status (0 is good):0
ffmpeg -y -i mbrl_v8_test9/zero_shot_data/bestMidWorst5.mp4 -vf drawtext="fontsize=50:fontfile=FreeSerif.ttf:text=''Turn in place ↲'':x=(w-text_w)/2:y=5" mbrl_v8_test9/zero_shot_data/bestMidWorst_txt5.mp4
Status (0 is good):0
Done


In [24]:
## concatentate videos for all goals

print('Concatenating videos')
vidlist = os.path.join(data_subfolder,'vidlist.txt')
# create a list of the video names for ffmpeg
with open(vidlist, 'w') as f:
    for i in range(6):
        end_str = str(i) + '.mp4'
#         vidpath = 'output_unseen'+end_str # ffmpeg seems to pick up the paths from the location of the file
        vidpath = tag+'_txt'+end_str # ffmpeg seems to pick up the paths from the location of the file
        f.write("file '" +vidpath +"'\n")
# run concat command in ffmpeg
run_str2 = 'ffmpeg -f concat -safe 0 -i ' + vidlist + ' -c copy ' + os.path.join(data_subfolder,tag+'.mp4')
run_str2 = run_str2 + ' -y'
status = os.system(run_str2)
print('Status (0 is good):' + str(status))
print('Done')

Concatenating videos
Status (0 is good):0
Done


In [25]:
# Add black bars separating the videos
# http://ffmpeg.org/ffmpeg-filters.html#drawbox
# ffmpeg -y -i sample.mp4 -vf "drawbox=x=10:y=10:w=100:h=100:color=pink@0.5:t=fill"  outputfile.mp4
data_subfolder  ='/home/cobracommander/modular_mbrl/mbrl_v8_test9/zero_shot_data'
import os

run_str = 'ffmpeg -y -i '
run_str += os.path.join(data_subfolder,tag+'.mp4')
# run_str += ' -vf "drawbox=x=10:y=10:w=100:h=100:color=black@0.5:t=fill"  '
run_str += ' -vf "'
# filters, separated by a comma
# 2048 x 1152 video total size, 2048/4 = 512, 1152/3 = 384
run_str += 'drawbox=x=0:y=0:w=512:h=1152:color=black:thickness=1, '
run_str += 'drawbox=x=512:y=0:w=512:h=1152:color=black:thickness=1, '
run_str += 'drawbox=x=1024:y=0:w=512:h=1152:color=black:thickness=1, '
run_str += 'drawbox=x=1536:y=0:w=512:h=1152:color=black:thickness=1, '
run_str += 'drawbox=x=0:y=0:w=2048:h=384:color=black:thickness=1, '
run_str += 'drawbox=x=0:y=384:w=2048:h=384:color=black:thickness=1, '
run_str += 'drawbox=x=0:y=768:w=2048:h=384:color=black:thickness=1'
run_str += '"  '
run_str += os.path.join(data_subfolder,tag+'_bars.mp4')
print(run_str)
status = os.system(run_str)
print('Status (0 is good):' + str(status))
print('Done')

ffmpeg -y -i /home/cobracommander/modular_mbrl/mbrl_v8_test9/zero_shot_data/bestMidWorst.mp4 -vf "drawbox=x=0:y=0:w=512:h=1152:color=black:thickness=1, drawbox=x=512:y=0:w=512:h=1152:color=black:thickness=1, drawbox=x=1024:y=0:w=512:h=1152:color=black:thickness=1, drawbox=x=1536:y=0:w=512:h=1152:color=black:thickness=1, drawbox=x=0:y=0:w=2048:h=384:color=black:thickness=1, drawbox=x=0:y=384:w=2048:h=384:color=black:thickness=1, drawbox=x=0:y=768:w=2048:h=384:color=black:thickness=1"  /home/cobracommander/modular_mbrl/mbrl_v8_test9/zero_shot_data/bestMidWorst_bars.mp4
Status (0 is good):0
Done


In [17]:
# # write text overlay on video
# # ffmpeg -i input.mp4 -vf drawtext="fontfile=/path/to/font.ttf: \
# # text='Stack Overflow': fontcolor=white: fontsize=24: box=1: boxcolor=black@0.5: \
# # boxborderw=5: x=(w-text_w)/2: y=(h-text_h)/2" -codec:a copy output.mp4
# run_str = 'ffmpeg -y -i '
# run_str += os.path.join(data_subfolder,tag+'.mp4')
# run_str += ' -vf drawtext="fontsize=50:fontfile=FreeSerif.ttf:'
# run_str += "text=''Direction ➔'':"
# # run_str += 'x=(w-text_w)/2:y=(h-text_h)/2"'
# run_str += 'x=(w-text_w)/2:y=5" '
# run_str += os.path.join(data_subfolder,'output_text.mp4')
# print(run_str)
# status = os.system(run_str)
# print('Status (0 is good):' + str(status))
# print('Done')

ffmpeg -y -i mbrl_v8_test9/zero_shot_data/bestMidWorst.mp4 -vf drawtext="fontsize=50:fontfile=FreeSerif.ttf:text=''Direction ➔'':x=(w-text_w)/2:y=5" mbrl_v8_test9/zero_shot_data/output_text.mp4
Status (0 is good):0
Done


In [27]:
import torch
import os
import numpy as np
data_subfolder  ='/home/cobracommander/modular_mbrl/mbrl_v8_test9/zero_shot_data'

# sort the results and get the best 4, 4 near the mean, and 4 worst
data = torch.load(os.path.join(data_subfolder,'transfer_results.ptx'))
print([key for key in data])
# print(data['urdf_names'])
unseen_inds = np.array(data['unseen_inds'])
num_unseen =len(unseen_inds) 
vel_all = np.array(data['vel_metric_list'])
vel_unseen = vel_all[unseen_inds]
sort_inds = np.argsort(vel_unseen)
top4 = unseen_inds[sort_inds[:4]]
bottom4 = unseen_inds[sort_inds[-4:]]
# bottom4 = unseen_inds[sort_inds[-5:-1]]# first one is clone of second
mid_ind = int(np.floor(num_unseen/2))
mid4 = unseen_inds[sort_inds[mid_ind-2:mid_ind+2]]
selected_inds = np.concatenate([top4, mid4, bottom4])
urdf_selected = [data['urdf_names'][i] for i in selected_inds]
vel_selected = [data['vel_metric_list'][i] for i in selected_inds]
print(urdf_selected)
print(vel_selected)

['unseen_inds', 'seen_inds', 'urdf_names', 'vel_metric_list', 'vel_baseline_list']
['wnlllw', 'wnwwlw', 'wwwwnw', 'lnwwll', 'llwlwl', 'wwllnw', 'lnwlwl', 'llllww', 'lnlwnl', 'lnlwwl', 'wnwlnl', 'lnllwl']
[0.029497588537774524, 0.02963361268776923, 0.029779535947246402, 0.03184932144364402, 0.042081060643168945, 0.042433678802401086, 0.04272666322161037, 0.0429001386159886, 0.07488818860911474, 0.07963283258693077, 0.0821855855934905, 0.08686433297338465]


In [10]:
run_str

'ffmpeg -i mbrl_v8_test9/zero_shot_data/wnwwlw_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/wwwwnw_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/lnwwll_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/wwwwll_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/llwlwl_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/wwllnw_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/lnwlwl_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/llllww_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/lnlwwl_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/wnwlnl_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/lnllwl_goal5.mp4 -i mbrl_v8_test9/zero_shot_data/lwllnl_goal5.mp4 -filter_complex " [0:v] setpts=4.855263157894737*PTS, scale=512x384 [a0]; [1:v] setpts=4.92*PTS, scale=512x384 [a1]; [2:v] setpts=5.3478260869565215*PTS, scale=512x384 [a2]; [3:v] setpts=3.84375*PTS, scale=512x384 [a3]; [4:v] setpts=3.1810344827586206*PTS, scale=512x384 [a4]; [5:v] setpts=4.054945054945055*PTS, scale=512x384 [a5]; [6:v] setpts=3.69*PTS, scale=512x384 [a6]; [7:v] setpts=3.0*PTS, scale=

In [20]:
# # # # video tile compile. This works ok
# urdf_vids = ['llllll_apply_policy.mp4',
#              'wnwwnw_apply_policy.mp4',
#              'llwwll_apply_policy.mp4',
#              'lnwwnl_apply_policy.mp4']
# vid_frames = []
# import subprocess
# for urdf_vid in urdf_vids:
#     batcmd = ('ffprobe -select_streams v -show_streams '
#            + os.path.join(data_subfolder,urdf_vid))
#     result = str(subprocess.check_output(batcmd, shell=True))
#     nb_frames_index = result.find('nb_frames=')
#     next_backslash = result[nb_frames_index:].find('\\n')
#     n_frames = result[nb_frames_index+len('nb_frames='):
#                       nb_frames_index+next_backslash]
#     vid_frames.append(int(n_frames))

# # calculate retiming
# print(np.array(vid_frames)/np.max(vid_frames))
# retiming = 2*np.max(vid_frames)/np.array(vid_frames)

# run_str = ('ffmpeg'+
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[0])+
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[1])+ 
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[2])+
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[3])+ 
#            ' -filter_complex '+ 
#            '"nullsrc=size=2048x1536 [base]; '+
#            '[0:v] setpts=' + str(retiming[0]) +'*PTS, scale=1024x768 [upperleft]; '+
#            '[1:v] setpts=' + str(retiming[1]) +'*PTS, scale=1024x768 [upperright]; '+
#            '[2:v] setpts=' + str(retiming[2]) +'*PTS, scale=1024x768 [lowerleft]; '+
#            '[3:v] setpts=' + str(retiming[3]) +'*PTS, scale=1024x768 [lowerright]; '+
#            '[base][upperleft] overlay=shortest=1 [tmp1]; '+
#            '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
#            '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
#            '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768" ' +
#            os.path.join(data_subfolder,'output.mp4')
#           )

# # run_str = ('ffmpeg'+
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[0])+
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[1])+ 
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[2])+
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[3])+ 
# #            ' -filter_complex '+ 
# #            '"nullsrc=size=2048x1536 [base]; '+
# #            '[0:v] setpts=PTS-STARTPTS, scale=1024x768 [upperleft]; '+
# #            '[1:v] setpts=PTS-STARTPTS, scale=1024x768 [upperright]; '+
# #            '[2:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerleft]; '+
# #            '[3:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerright]; '+
# #            '[base][upperleft] overlay=shortest=1 [tmp1]; '+
# #            '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
# #            '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
# #            '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768" ' +
# #            os.path.join(data_subfolder,'output.mp4')
# #           )
# status = os.system(run_str)
# print(status)
# if status != 0:
#     run_str = run_str + ' -y'
#     status = os.system(run_str)
#     print(status)



# # ffmpeg -i test.mp4 -i test.mp4 -i test2.mp4 -i test2.mp4 -filter_complex 
# # "nullsrc=size=2048x1536 [base]; [0:v] setpts=PTS-STARTPTS, scale=1024x768 [upperleft]; [1:v] setpts=PTS-STARTPTS, scale=1024x768 [upperright]; [2:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerleft]; [3:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; [tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; [tmp3][lowerright] overlay=shortest=1:x=1024:y=768" output.mp4

In [8]:

# import subprocess
# # run_str = ['ffmpeg',
# #            '-i', os.path.join(data_subfolder, urdf_vids[0]), 
# #            '-i', os.path.join(data_subfolder, urdf_vids[1]), 
# #            '-i', os.path.join(data_subfolder, urdf_vids[2]), 
# #            '-i', os.path.join(data_subfolder, urdf_vids[3]), 
# #            '-filter_complex', 
# #            ('"nullsrc=size=2048x1536 [base]; '+
# #            '[0:v] setpts=PTS-STARTPTS, scale=1024x768 [upperleft]; '+
# #            '[1:v] setpts=PTS-STARTPTS, scale=1024x768 [upperright]; '+
# #            '[2:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerleft]; '+
# #            '[3:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerright]; '+
# #            '[base][upperleft] overlay=shortest=1 [tmp1]; '+
# #            '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
# #            '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
# #            '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768"'),
# #            'output.mp4'
# #             ]
# # cmd_status = subprocess.call(run_str)
# cmd_status = subprocess.call(['ffmpeg', '-i',
#                   os.path.join(data_subfolder,'output.mp4'), '-filter:v',
#                   'setpts=2.0*PTS',
#                   os.path.join(data_subfolder,'output_1x.mp4'), '-y']
# # print(cmd_status)
